Command Profile in TVB
----------------------

This is a simple demo showing for to interract with TVB in the command profile.

More examples can be found here:
https://github.com/the-virtual-brain/tvb-framework/tree/master/tvb/interfaces/command/demos

In [ ]:
from tvb.interfaces.command.lab import *

In [ ]:
list_projects()

Create a new project to test with

In [ ]:
import uuid
proj = new_project("sandbox" + str(uuid.uuid4()))

In [ ]:
list_projects()

If you have the web UI open at the same time, you'll notice there's now a new project named "sandbox_project"

In [ ]:
list_datatypes(proj.id)

Nothing! Let's import a connectivity

In [ ]:
import os
from tvb.datasets import TVBZenodoDataset
p = TVBZenodoDataset().fetch_data('connectivity_66.zip')
import_op = import_conn_zip(proj.id, p)

import_op = wait_to_finish(import_op)
import_op

In [ ]:
list_operation_results(import_op.id)

In [ ]:
conectivity_gid = get_operation_results(import_op.id)[0].gid

Now let's grab that connectivity from the database. 
IMPORTANT: Make sure to adjust the DT gid below to match the value in the id column above and run a simulation with it.

In [ ]:
simulator_model = SimulatorAdapterModel()
# ConnectivityIndex gid from the column above
simulator_model.connectivity = conectivity_gid
simulator_model.simulation_length = 100
sim_op = fire_simulation(proj.id, simulator_model)
sim_op = wait_to_finish(sim_op)
sim_op

We should wait for the simulation to finish, and afterwards, TimeSeries should also be part of this project.

Refresh the Web GUI to see the status of your operation.

In [ ]:
list_operation_results(sim_op.id)

In [ ]:
time_series_region_index_id = get_operation_results(sim_op.id)[1].id

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Use TimeSeriesRegionIndex id
ts  = load_dt(time_series_region_index_id)
for reg in range(66):
    plt.plot(ts.data[:, 0, reg, 0])
plt.show()

You can re-evaluate this cell multiple times while it's running to see how the results gradually show up as the simulation finishes.